# Bagging (explicitly)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.tree
import sklearn.metrics
import sklearn.model_selection

## First: create a dataset to explore

In [ ]:
x = np.linspace(0,10,100)

In [ ]:
np.random.seed(42)
noise = np.random.normal(0,1.5,100)

y = x + noise

In [ ]:
plt.plot(x,y,'ko')

## Straight-forward linear regression

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x.reshape(-1,1), y, random_state=42)

In [ ]:
lin_reg = sklearn.linear_model.LinearRegression()
lin_reg.fit(x_train,y_train)
print('R2 score: ',lin_reg.score(x_test, y_test))

In [ ]:
plt.plot(x_train,y_train,'ko')
plt.plot(x_test,y_test,'bo')
plt.plot([0,10],lin_reg.predict([[0],[10]]))

## Bootstrap samples and aggregation

Use bootstrapping to get training samples, then take average of the regressor's predictions for any given sample.

We will select `k` samples with replacement, train our linear regression algorithm, and then repeat the process `t` times.

In [ ]:
k = 5
t = 5

Since we need to select an element from `x_train` and `y_train` in tandem, we make choices among the list of indices.

In [ ]:
ix = range(0,len(x_train))

In [ ]:
np.random.choice(ix, k)

In [ ]:
newix = np.random.choice(ix, k)

In [ ]:
x[newix]

Here's where we repeat the training `t` times on samples with `k` elements chosen from `(x_train, y_train)`

In [ ]:
lin_regs = []
for i in range(t):
    
    newix = np.random.choice(ix, k)
    
    lin_regs.append(sklearn.linear_model.LinearRegression())
    
    lin_regs[i].fit(x_train[newix].reshape(-1,1),
                    y_train[newix])

We plot all of the linear fits:

In [ ]:
plt.plot(x,y,'ko')
x_edge = [[0],[10]]
for i in range(t):
    plt.plot(x_edge,lin_regs[i].predict(x_edge))

When looking at predictions, we take an average over all the predictors.

In [ ]:
plt.plot(x,y,'ko')

xnew = np.linspace(0,10,100)
ynew = []
for i in xnew:
    tmpx = [[i]]
    n = [j.predict(tmpx) for j in lin_regs]
    ynew.append(np.mean(n))

plt.plot(xnew,ynew,'b')
plt.plot([0,10],lin_reg.predict([[0],[10]]), 'g')

In [ ]:
y_pred = []
for i in x_test:
    n = [j.predict([i]) for j in lin_regs]
    y_pred.append(np.mean(n))
r2score = sklearn.metrics.r2_score(y_test, y_pred)
print('R2 score: ',r2score)

The above is very good, considering we only trained 5 times with 5 samples each time.

Repeat the above for other numbers.

# Decision Trees

We are going to repeat this process now using Decision Trees instead of Linear Regression.

First, we train one decision tree to see how it does.

In [ ]:
tree_reg = sklearn.tree.DecisionTreeRegressor()
tree_reg.fit(x_train,y_train)
print('R2 score: ',tree_reg.score(x_test, y_test))

In [ ]:
plt.plot(x_train,y_train,'ko')
plt.plot(x_test,y_test,'bo')
xnew = np.linspace(0,10,1000).reshape(-1,1)
ynew = tree_reg.predict(xnew)
plt.plot(xnew,ynew)

The decision tree here very much overfits to the training data.

We will see whether this can be improved by using bootstrapping.

In [ ]:
k = 5
t = 5

In [ ]:
tree_regs = []
for i in range(t):
    
    newix = np.random.choice(ix, k)
    
    tree_regs.append(sklearn.tree.DecisionTreeRegressor())
    
    tree_regs[i].fit(x_train[newix].reshape(-1,1),
                     y_train[newix])

In [ ]:
plt.plot(x_train,y_train,'ko')
plt.plot(x_test,y_test,'bo')
x_edge = np.linspace(0,10,1000).reshape(-1,1)
for i in range(t):
    plt.plot(x_edge,tree_regs[i].predict(x_edge))

The above shows the various decision trees trained on only `k` points.

For our aggregate prediction, we average the predictions over all of the `t` predictors.

In [ ]:
plt.plot(x_train,y_train,'ko')
plt.plot(x_test,y_test,'bo')

xnew = np.linspace(0,10,1000).reshape(-1,1)
ynew = []
for i in xnew:
    n = [j.predict([i]) for j in tree_regs]
    ynew.append(np.mean(n))

plt.plot(xnew,ynew)

In [ ]:
y_pred = []
for i in x_test:
    n = [j.predict([i]) for j in tree_regs]
    y_pred.append(np.mean(n))
r2score = sklearn.metrics.r2_score(y_test, y_pred)
print(r2score)

This model outperforms the single decision tree!